In [89]:
import requests
import numpy as np
import pandas as pd
import os

In [90]:
headers = {"user-agent":"disc - dj_danyp"}

# Requesting the data and formating to database

In [91]:
latest_data = requests.get("https://prices.runescape.wiki/api/v1/osrs/latest", headers=headers)
latest_data = latest_data.json()
hr24_data = requests.get("https://prices.runescape.wiki/api/v1/osrs/24h", headers=headers)
hr24_data = hr24_data.json()


In [92]:
#creating dataframe. old axis was item ID so we're renaming that and adding a new index.
latest_db = pd.DataFrame(latest_data["data"]).transpose().rename_axis("Item_id").reset_index()
latest_db["Item_id"] = latest_db["Item_id"].astype(int)

#changing column names and dropping time columns. We'll use previous hour volume to judge volume.
latest_db.rename(columns = {"high" : "high_price", "low" : "low_price"}, inplace=True)
latest_db.drop(["highTime", "lowTime"], axis=1, inplace=True)

latest_db

,Item_id,high_price,low_price
0,2,163.0,161.0
1,6,195020.0,188751.0
2,8,190681.0,185297.0
3,10,204850.0,190016.0
4,12,204876.0,194501.0
...,...,...,...
3882,28179,3817.0,2920.0
3883,28181,41498.0,27225.0
3884,28184,6000.0,4995.0
3885,28190,110000.0,41526.0


In [93]:
#creating dataframe. old axis was item ID so we're renaming that and adding a new index.
hr24_db = pd.DataFrame(hr24_data["data"]).transpose().rename_axis("Item_id").reset_index()
hr24_db["Item_id"] = hr24_db["Item_id"].astype(int)

hr24_db.drop(["avgHighPrice", "avgLowPrice"], axis=1, inplace=True)

hr24_db

,Item_id,highPriceVolume,lowPriceVolume
0,2,20724099.0,8383035.0
1,6,96.0,348.0
2,8,78.0,343.0
3,10,90.0,438.0
4,12,82.0,322.0
...,...,...,...
3657,28179,4458.0,1398.0
3658,28181,30.0,51.0
3659,28184,6760.0,1615.0
3660,28190,28.0,74.0


In [94]:
cwd = os.getcwd()
item_ids = pd.read_excel(cwd + r"\Item_id.xlsx")
item_ids

,Item,Item_id
0,3rd age 2h axe,28226
1,3rd age amulet,10344
2,3rd age axe,20011
3,3rd age bow,12424
4,3rd age cloak,12437
...,...,...
3959,Zogre bones,4812
3960,Zombie head (Treasure Trails),19912
3961,Zul-andra teleport,12938
3962,Zulrah's scales,12934


# Merging the databases

In [95]:
latest_db = pd.merge(item_ids, latest_db, on = "Item_id", how = "left").sort_values("Item_id")
latest_db

,Item,Item_id,high_price,low_price
878,Cannonball,2,163.0,161.0
875,Cannon base,6,195020.0,188751.0
877,Cannon stand,8,190681.0,185297.0
874,Cannon barrels,10,204850.0,190016.0
876,Cannon furnace,12,204876.0,194501.0
...,...,...,...,...
2790,Rune 2h axe,28214,NaN,NaN
1132,Dragon 2h axe,28217,NaN,NaN
1010,Crystal 2h axe,28220,NaN,NaN
1011,Crystal 2h axe (inactive),28223,NaN,NaN


In [96]:
item_db = pd.merge(latest_db, hr24_db, on = "Item_id", how = "left").sort_values("Item_id")
item_db

,Item,Item_id,high_price,low_price,highPriceVolume,lowPriceVolume
0,Cannonball,2,163.0,161.0,20724099.0,8383035.0
1,Cannon base,6,195020.0,188751.0,96.0,348.0
2,Cannon stand,8,190681.0,185297.0,78.0,343.0
3,Cannon barrels,10,204850.0,190016.0,90.0,438.0
4,Cannon furnace,12,204876.0,194501.0,82.0,322.0
...,...,...,...,...,...,...
3959,Rune 2h axe,28214,NaN,NaN,NaN,NaN
3960,Dragon 2h axe,28217,NaN,NaN,NaN,NaN
3961,Crystal 2h axe,28220,NaN,NaN,NaN,NaN
3962,Crystal 2h axe (inactive),28223,NaN,NaN,NaN,NaN


In [97]:
item_db["totalVolume"] = item_db["highPriceVolume"] + item_db["lowPriceVolume"]
item_db

,Item,Item_id,high_price,low_price,highPriceVolume,lowPriceVolume,totalVolume
0,Cannonball,2,163.0,161.0,20724099.0,8383035.0,29107134.0
1,Cannon base,6,195020.0,188751.0,96.0,348.0,444.0
2,Cannon stand,8,190681.0,185297.0,78.0,343.0,421.0
3,Cannon barrels,10,204850.0,190016.0,90.0,438.0,528.0
4,Cannon furnace,12,204876.0,194501.0,82.0,322.0,404.0
...,...,...,...,...,...,...,...
3959,Rune 2h axe,28214,NaN,NaN,NaN,NaN,NaN
3960,Dragon 2h axe,28217,NaN,NaN,NaN,NaN,NaN
3961,Crystal 2h axe,28220,NaN,NaN,NaN,NaN,NaN
3962,Crystal 2h axe (inactive),28223,NaN,NaN,NaN,NaN,NaN


# Potion arbitrage

## Running calculations / creating new database

In [98]:
#Find the most eligible potions.

Top_10_potions = item_db[(item_db.highPriceVolume > 1000) &\
                            (item_db.lowPriceVolume > 1000) &\
                                (item_db.low_price > 1000) &\
                                    (item_db.Item.str.contains("\(3\)")) &\
                                        (~item_db.Item.str.contains("Divine|Blighted"))]

Top_10_potions = Top_10_potions.sort_values("highPriceVolume", ascending=False)[:10]
Top_10_potions

,Item,Item_id,high_price,low_price,highPriceVolume,lowPriceVolume,totalVolume
989,Super energy(3),3018,3184.0,3117.0,74160.0,137818.0,211978.0
55,Prayer potion(3),139,6860.0,6860.0,55901.0,385069.0,440970.0
993,Super restore(3),3026,7194.0,7220.0,49121.0,339784.0,388905.0
1815,Saradomin brew(3),6687,7500.0,7633.0,43466.0,370940.0,414406.0
70,Ranging potion(3),169,1165.0,1005.0,35627.0,138519.0,174146.0
67,Super defence(3),163,3020.0,2973.0,25494.0,181421.0,206915.0
64,Super strength(3),157,2812.0,2795.0,23260.0,124544.0,147804.0
3449,Bastion potion(3),22464,24105.0,24102.0,7504.0,8531.0,16035.0
2856,Stamina potion(3),12627,7800.0,7499.0,5954.0,13049.0,19003.0
2864,Super combat potion(3),12697,10400.0,9726.0,2594.0,5267.0,7861.0


### Above is the list of potions we will use. The price and volume are high enough that we should see a large profit with low risk.

In [99]:
potion_list = Top_10_potions[:10][["Item", "Item_id"]]

#stripping the (3) in order to look up 4 dose potions
potion_list["Item"] = potion_list["Item"].apply(lambda x: x.replace("(3)", ""))



In [100]:
#Looks up 3 dose ID and renames columns
potion_list["4_dose"] = potion_list["Item"].apply(lambda x: item_db.loc[item_db["Item"] == (x + "(4)"), 'Item_id'].iloc[0])
potion_list.rename(columns={"Item_id": "3_dose"}, inplace = True)
potion_list

,Item,3_dose,4_dose
989,Super energy,3018,3016
55,Prayer potion,139,2434
993,Super restore,3026,3024
1815,Saradomin brew,6687,6685
70,Ranging potion,169,2444
67,Super defence,163,2442
64,Super strength,157,2440
3449,Bastion potion,22464,22461
2856,Stamina potion,12627,12625
2864,Super combat potion,12697,12695


In [101]:
#getting potion names as a list
potions_to_analyze = potion_list["Item"].to_list()


In [104]:
#creating a function that will create the database given the potion list
def potion_arb_sheet(potions_to_analyze):

    potion_analyzed = []

    instant = []
    slow = [] 

    d3_bidask = []
    d4_bidask = []

    d3_volume = []
    d4_volume = []





    for potion in potions_to_analyze:
       
        #using potion name to look up item ids in potion_list
        id_3_dose = potion_list.loc[potion_list['Item'] == potion, '3_dose'].values[0]
        id_4_dose = potion_list.loc[potion_list['Item'] == potion, '4_dose'].values[0]
        
        #using item ids to look up bid/ask spread in the item_db
        d4_high, d4_low = item_db.loc[item_db["Item_id"] == id_4_dose, ["high_price", "low_price"]].iloc[0]
        d3_high, d3_low = item_db.loc[item_db["Item_id"] == id_3_dose, ["high_price", "low_price"]].iloc[0]


        
        
        
        potion_analyzed.append(potion)

        instant.append(round(d4_low / 4 * .99 - d3_high / 3))
        slow.append(round(d4_high / 4 * .99 - d3_low / 3))
    
        d3_bidask.append(f"{round(d3_low)} - {round(d3_high)}")
        d4_bidask.append(f"{round(d4_low)} - {round(d4_high)}")

        d3_volume.append(round(item_db.loc[item_db["Item_id"] == id_3_dose, "totalVolume"].values[0]))
        d4_volume.append(round(item_db.loc[item_db["Item_id"] == id_4_dose, "totalVolume"].values[0]))

        




    
    return pd.DataFrame({'Potion': potion_analyzed,\
                            'Instant Profit/dose': instant,\
                            'Slow Profit/dose': slow,\
                                    "D3_Bid-Ask_Spread": d3_bidask,\
                                    "D4_Bid-Ask_Spread": d4_bidask,\
                                            "d3_volume": d3_volume,\
                                            "d4_volume": d4_volume})




,Potion,Instant Profit/dose,Slow Profit/dose,D3_Bid-Ask_Spread,D4_Bid-Ask_Spread,d3_volume,d4_volume
0,Super energy,-45,0,3117 - 3184,4108 - 4200,211978,344491
1,Prayer potion,21,32,6860 - 6860,9322 - 9369,440970,1319472
2,Super restore,34,45,7220 - 7194,9826 - 9905,388905,931602
3,Saradomin brew,15,-1,7633 - 7500,10160 - 10277,414406,1128526
4,Ranging potion,17,42,1005 - 1165,1639 - 1523,174146,676682
5,Super defence,114,148,2973 - 3020,4527 - 4602,206915,464276
6,Super strength,47,61,2795 - 2812,3976 - 4011,147804,410518
7,Bastion potion,210,211,24102 - 24105,33314 - 33314,16035,40606
8,Stamina potion,-97,37,7499 - 7800,10115 - 10248,19003,640736
9,Super combat potion,-169,86,9726 - 10400,13323 - 13447,7861,519234


## Using the function to create database

### Instant Profit/dose - The idea is that to lock in instant profit you'll buy at the asking price, decant the potions, and sell at the asking price. The "instant Profit/dose" shows how much you will profit per dose using the latest price recorded.

### Slow Profit/dose - You'll place a bid just above the d3_ask price and wait until your order is complete. Then you'll place an ask price just below D4_Ask. This works best with extreemly high volume items as the orders will fill faster

In [105]:
potion_arb_sheet(potions_to_analyze)

,Potion,Instant Profit/dose,Slow Profit/dose,D3_Bid-Ask_Spread,D4_Bid-Ask_Spread,d3_volume,d4_volume
0,Super energy,-45,0,3117 - 3184,4108 - 4200,211978,344491
1,Prayer potion,21,32,6860 - 6860,9322 - 9369,440970,1319472
2,Super restore,34,45,7220 - 7194,9826 - 9905,388905,931602
3,Saradomin brew,15,-1,7633 - 7500,10160 - 10277,414406,1128526
4,Ranging potion,17,42,1005 - 1165,1639 - 1523,174146,676682
5,Super defence,114,148,2973 - 3020,4527 - 4602,206915,464276
6,Super strength,47,61,2795 - 2812,3976 - 4011,147804,410518
7,Bastion potion,210,211,24102 - 24105,33314 - 33314,16035,40606
8,Stamina potion,-97,37,7499 - 7800,10115 - 10248,19003,640736
9,Super combat potion,-169,86,9726 - 10400,13323 - 13447,7861,519234
